In [13]:
import sys
from pathlib import Path
import torch

ROOT_PATH=Path("..").resolve().absolute()
DEVICE=torch.device("cpu")
sys.path.append(str(ROOT_PATH))

%load_ext autoreload
%autoreload 2
from evaluation.evaluate_separation import evaluate_tracks_chunks_simplified

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
!pwd

In [14]:
results = evaluate_tracks_chunks_simplified(Path("/home/irene/Documents/audio-diffusion-pytorch-trainer/separations/complete_context_slakh_4stems_resamples"), 
                                                orig_sr=22050, eps=1e-8,
                                                compute_likelihood=False)

1009it [00:11, 85.50it/s]

set()


3140it [01:53, 27.60it/s]


1    15.600772
2    16.930175
3    16.525742
4    13.971500
dtype: float64

In [16]:
results

,1,2,3,4
0,20.414639,23.992584,50.781300,18.528038
1,25.069036,27.335058,40.029137,22.979305
2,23.697376,27.729813,41.224686,24.840891
3,17.207308,26.189274,41.255264,20.025192
4,15.193639,24.048454,18.454071,13.510019
...,...,...,...,...
15463,26.268805,21.950933,40.784618,38.366982
15464,28.547485,21.126476,40.957554,38.461723
15465,29.054516,18.115562,40.237221,37.908806
15466,30.138922,16.412724,40.304409,37.851566
